## 20. Условная оптимизация. Метод штрафных функций

Рассмотрим один из многочисленных вариантов *метода штрафных функций*.
Задача условной оптимизации
$$
   f(x) \to \min
$$ 
при ограничениях 
$$
g_1(x) = 0, \dots, g_m(x) = 0, \quad
h_1(x) \le 0, \dots, h_p(x) \le 0
$$
решается при помощи решения последовательности вспомогательных задач безусловной оптимизации:
$$
\left(F(x) + c\sum_{i=1}^m g(h_i)^2 + c\sum_{k=1}^p \left[ h(h_k) \right]_+^2\right) \to \min,
$$
где $c$ – некоторая возрастающая последовательность (например, геометрическая прогрессия),
$[\alpha]_+ = \max\{0,\,\alpha\}$.

Реализуйте данный метод. Для решения задачи безусловной оптимизации воспользуйтесь функциями из библиотеки SciPy.

Проиллюстрируйте работу метода на двумерной и многомерных функциях Розенброка
$$
f(x_1,x_2,\dots,x_n) = \sum_{i=1}^{n-1} \left(  (1-x_i)^2+ 100 (x_{i+1} - x_i^2 )^2 \right) 
$$
с линейными и квадратичными ограничениями.
Для $n=2$ изобразите найденные точки минимума на каждой итерации.

In [1]:
from scipy.optimize import minimize, rosen
rz = lambda x: (1-x[0])**2 + 100*(x[1] - x[0]**2)**2;#Функция Розенброка
h_1 = lambda x: (x[0] - 2 * x[1] + 2);#
h_2 = lambda x: (-x[0] - 2 * x[1] + 6);#
h_3 = lambda x: (-x[0] + 2 * x[1] + 2);#Параметры исходя из ограничений

x0 = [2.3, 5];#Точки минимума и максимума
cons = ({'type': 'ineq', 'fun': h_1},
       {'type': 'ineq', 'fun': h_2},
       {'type': 'ineq', 'fun': h_3}) 
print(minimize(rz, x0, constraints=cons))#Функция расчёта функции Розенберга

x_c = [2.3, 3]
i = 1# Номер итерации
r = 1
b = 0.2
eps = 0.01 #Точность вычислений
while i < 1000:
    curr_func = lambda x: rz(x) + r*(1.0/(h_1(x)**2 + h_2(x)**2 + h_3(x)**2))#Метод штрафных функций
    x_c = minimize(curr_func, x_c).x;
    i += 1
    r  *= b;
    if curr_func(x_c) < eps:#Условие достижения определённой точности
        break
print(x_c)
print(i)

     fun: 9.406138556671796e-08
     jac: array([ 0.00408084, -0.00232047])
 message: 'Optimization terminated successfully.'
    nfev: 73
     nit: 17
    njev: 17
  status: 0
 success: True
       x: array([0.99971613, 0.99942073])
[0.99495003 0.98991398]
3


In [ ]:
Отчёт ещё делаю